# Title
[]()

In [20]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *

In [3]:
path = '/home/silvhua/repositories/GHL-chat/src/app/private'
contacts = load_json('contacts.json', path)
contactId = contacts['me_fb']

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *
from chat_functions import *

In [4]:
system_message_dict = dict()
conversation_dict = dict()
reply_dict = dict()
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()

In [4]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')
response = refresh_token()

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


# Initialize

In [5]:
response_dict = dict()

# Iteration 1

In [8]:
iteration = 1
payload = {
    "type": "FB",
    "message": append_timestamp('test message '),
}

response_dict[iteration] = ghl_request(
    contactId=contactId, 
    text=None,
    endpoint='sendMessage', 
    payload=payload, 
    location='CoachMcloone'
)
print(f'sendMessage: {response_dict[iteration]}')

input payload: {'type': 'FB', 'message': 'test message _2024-02-09_0814'}
Status code 201: Created
sendMessage: {'conversationId': '8bcsOnIFz1ZXiBYKfkkv', 'messageId': 'R4UjY0dYshiOtaI4310m', 'traceId': '28e8d633-bf29-4f67-831c-6c15dab13f75', 'status_code': 201, 'response_reason': 'Created'}


# Test conditions

In [9]:
messages_to_ignore = [
    'GET STARTED', '🍎 Nutrition', '💪 Training', '🧠 Knowledge'
]
'🍎 Nutrition' in messages_to_ignore

True

In [14]:
contact_tags = [
    'money_magnet_lead', 'chatgpt', 
    'no chatbot',
    'money_magnet_schedule'
]
tags_to_ignore = [
    'no chatbot',
    'money_magnet_schedule'
]

print(set(contact_tags).intersection(set(tags_to_ignore)) == None)
print(set(contact_tags).intersection(set(tags_to_ignore)))
print(len(set(contact_tags).intersection(set(tags_to_ignore))))
if set(contact_tags).intersection(set(tags_to_ignore)):
    print('Common items found')
else:
    print('No common items found')

False
{'money_magnet_schedule', 'no chatbot'}
2
Common items found


# Test task creation

In [8]:
results_dict = dict()

In [9]:
import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
import os
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import random
import boto3
try:
    from dotenv import load_dotenv
    load_dotenv(dotenv_path='../src/.env')
except:
    pass

## iteration 1

In [26]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = '80pzXdC3MCqKRpzrteuj'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations'):  
            endpoint_url = f'conversations/search' if endpoint=='searchConversations' else f'contacts/' 
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1
results_dict[iteration] = ghl_request(
    contactId=contactId, 
    endpoint='createTask', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone'
)
results_dict[iteration]

Status code 422: Unprocessable Entity


{'status': 422,
 'message': 'The assigned to field is invalid.',
 'traceId': 'f627eb80-e1e5-4188-8ddb-9f2365f8a9e7',
 'status_code': 422,
 'response_reason': 'Unprocessable Entity'}

## 1.1

In [27]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = 'Silvia Hua'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations'):  
            endpoint_url = f'conversations/search' if endpoint=='searchConversations' else f'contacts/' 
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.1
results_dict[iteration] = ghl_request(
    contactId=contactId, 
    endpoint='createTask', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone'
)
results_dict[iteration]

Status code 422: Unprocessable Entity


{'status': 422,
 'message': 'The assigned to field is invalid.',
 'traceId': '1386a9e4-4cb0-4850-b67b-a94f84ed836f',
 'status_code': 422,
 'response_reason': 'Unprocessable Entity'}

## 1.2

In [30]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = 'Silvia Hua'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations') | (endpoint == 'searchUsers'): 
            if endpoint=='searchConversations':
                endpoint_url = f'conversations/search'
            elif endpoint == 'searchUsers':
                endpoint_url = f'users/search'            
            else:
                endpoint_url = f'contacts/' 
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.2
# results_dict[iteration] = ghl_request(
#     contactId=contactId, 
#     endpoint='createTask', 
#     params_dict=None,
#     payload=None, 
#     location='CoachMcloone'
# )
results_dict[iteration] = ghl_request(
    contactId='Silvia Hua', 
    endpoint='searchUsers', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone',
    locationId = 'fsdVH26v30hoBBQOBttG'
)
results_dict[iteration]

Status code 422: Unprocessable Entity


{'statusCode': 422,
 'message': ['companyId must be a string', 'companyId should not be empty'],
 'error': 'Unprocessable Entity',
 'traceId': '63e7d0ae-9e9b-469a-857e-bd7c13fa938c',
 'status_code': 422,
 'response_reason': 'Unprocessable Entity'}

## 1.3

In [31]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = 'Silvia Hua'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations') | (endpoint == 'searchUsers') | (endpoint == 'getLocation'): 
            if endpoint=='searchConversations':
                endpoint_url = f'conversations/search'
            elif endpoint == 'searchUsers':
                endpoint_url = f'users/search'            
            elif endpoint == 'getContacts':
                endpoint_url = f'contacts/'
            else:
                endpoint_url = f'locations/{locationId}'
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.3
# results_dict[iteration] = ghl_request(
#     contactId=contactId, 
#     endpoint='createTask', 
#     params_dict=None,
#     payload=None, 
#     location='CoachMcloone'
# )
# results_dict[iteration] = ghl_request(
#     contactId='Silvia Hua', 
#     endpoint='searchUsers', 
#     params_dict=None,
#     payload=None, 
#     location='CoachMcloone',
#     locationId = 'fsdVH26v30hoBBQOBttG'
results_dict[iteration] = ghl_request(
    contactId='Silvia Hua', 
    endpoint='getLocation', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone',
    locationId = 'fsdVH26v30hoBBQOBttG'
)
results_dict[iteration]

Status code 401: Unauthorized


{'statusCode': 401,
 'message': 'The token is not authorized for this scope.',
 'error': 'Unauthorized',
 'status_code': 401,
 'response_reason': 'Unauthorized'}

# Refresh token

In [33]:
from ghl_requests import refresh_token
iteration = 1.4
results_dict[iteration] = refresh_token()

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


In [39]:
from data_functions import parse_json_string
tokens_dict = parse_json_string(results_dict[iteration]['body'])
tokens_dict

{'access_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdXRoQ2xhc3MiOiJMb2NhdGlvbiIsImF1dGhDbGFzc0lkIjoiZnNkVkgyNnYzMGhvQkJRT0J0dEciLCJzb3VyY2UiOiJJTlRFR1JBVElPTiIsInNvdXJjZUlkIjoiNjRjMGFkMjU1MjdjM2YxZWU5MGJjNTE3LWxramE0dGoyIiwiY2hhbm5lbCI6Ik9BVVRIIiwicHJpbWFyeUF1dGhDbGFzc0lkIjoiZnNkVkgyNnYzMGhvQkJRT0J0dEciLCJvYXV0aE1ldGEiOnsic2NvcGVzIjpbImNvbnZlcnNhdGlvbnMvbWVzc2FnZS5yZWFkb25seSIsImNvbnZlcnNhdGlvbnMvbWVzc2FnZS53cml0ZSIsInVzZXJzLnJlYWRvbmx5IiwiY29udmVyc2F0aW9ucy5yZWFkb25seSIsImNvbnRhY3RzLndyaXRlIiwiY29udGFjdHMucmVhZG9ubHkiLCJ3b3JrZmxvd3MucmVhZG9ubHkiLCJjYWxlbmRhcnMvZXZlbnRzLnJlYWRvbmx5IiwiY2FsZW5kYXJzL2V2ZW50cy53cml0ZSIsImNhbGVuZGFycy53cml0ZSIsImNhbGVuZGFycy9ncm91cHMucmVhZG9ubHkiLCJjYWxlbmRhcnMvZ3JvdXBzLndyaXRlIiwiY29udmVyc2F0aW9ucy53cml0ZSIsImZvcm1zLnJlYWRvbmx5IiwiZm9ybXMud3JpdGUiLCJsaW5rcy5yZWFkb25seSIsImxvY2F0aW9ucy90YXNrcy53cml0ZSIsImxvY2F0aW9ucy90YXNrcy5yZWFkb25seSIsImxvY2F0aW9ucy90YWdzLndyaXRlIiwibG9jYXRpb25zL3RhZ3MucmVhZG9ubHkiLCJidXNpbmVzc2VzLnJlYWRvbmx5IiwiY2FsZW5kYXJzLnJlY

## 1.5

In [42]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = 'Silvia Hua'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations') | (endpoint == 'searchUsers') | (endpoint == 'getLocation'): 
            if endpoint=='searchConversations':
                endpoint_url = f'conversations/search'
            elif endpoint == 'searchUsers':
                endpoint_url = f'users/search'            
            elif endpoint == 'getContacts':
                endpoint_url = f'contacts/'
            else:
                endpoint_url = f'locations/{locationId}'
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.5
# results_dict[iteration] = ghl_request(
#     contactId=contactId, 
#     endpoint='createTask', 
#     params_dict=None,
#     payload=None, 
#     location='CoachMcloone'
# )
params_dict = {
    'companyId': 'n317fV7kZWH4j5BFN6g9',
    'query': 'Silvia Hua',
    'locationId': 'fsdVH26v30hoBBQOBttG'
}
results_dict[iteration] = ghl_request(
    contactId='Silvia Hua', 
    endpoint='searchUsers', 
    params_dict=params_dict,
    payload=None, 
    location='CoachMcloone'
)
results_dict[iteration]

Status code 200: OK


{'users': [{'id': 'qFootuYP7QMGCn6qXDXp',
   'deleted': False,
   'email': 'silvhua@gmail.com',
   'extension': '',
   'firstName': 'Silvia',
   'lastName': 'Hua',
   'name': 'Silvia Hua',
   'permissions': {'adwordsReportingEnabled': True,
    'affiliateManagerEnabled': True,
    'agentReportingEnabled': True,
    'appointmentsEnabled': True,
    'assignedDataOnly': False,
    'attributionsReportingEnabled': True,
    'bloggingEnabled': True,
    'botService': False,
    'bulkRequestsEnabled': True,
    'campaignsEnabled': True,
    'campaignsReadOnly': False,
    'cancelSubscriptionEnabled': True,
    'communitiesEnabled': True,
    'contactsEnabled': True,
    'contentAiEnabled': True,
    'conversationsEnabled': True,
    'dashboardStatsEnabled': True,
    'facebookAdsReportingEnabled': True,
    'funnelsEnabled': True,
    'invoiceEnabled': True,
    'leadValueEnabled': True,
    'marketingEnabled': True,
    'membershipEnabled': True,
    'onlineListingsEnabled': True,
    'oppor

In [43]:
iteration = 1.51
results_dict[iteration] = ghl_request(
    contactId=contactId, 
    endpoint='createTask', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone'
)

Status code 400: Bad Request


In [44]:
results_dict[iteration]

{'error': 'Contact with id qFootuYP7QMGCn6qXDXp not found',
 'status_code': 400,
 'response_reason': 'Bad Request'}

## 1.6

In [45]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = 'qFootuYP7QMGCn6qXDXp'
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations') | (endpoint == 'searchUsers') | (endpoint == 'getLocation'): 
            if endpoint=='searchConversations':
                endpoint_url = f'conversations/search'
            elif endpoint == 'searchUsers':
                endpoint_url = f'users/search'            
            elif endpoint == 'getContacts':
                endpoint_url = f'contacts/'
            else:
                endpoint_url = f'locations/{locationId}'
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.5
results_dict[iteration] = ghl_request(
    contactId=contactId, 
    endpoint='createTask', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone'
)

# results_dict[iteration] = ghl_request(
#     contactId='Silvia Hua', 
#     endpoint='searchUsers', 
#     params_dict=params_dict,
#     payload=None, 
#     location='CoachMcloone'
# )
results_dict[iteration]

Status code 201: Created


{'task': {'id': 'K1A7i2tSNcD5DowHF5QZ',
  'title': 'Send message to contact 6kIAuqeiBUxJ3nWTwNVE.',
  'body': 'Test task via GHL API at 2024-02-10T01:38:25Z Pacific time',
  'assignedTo': 'qFootuYP7QMGCn6qXDXp',
  'dueDate': '2024-02-10T01:38:25.000Z',
  'completed': False,
  'contactId': '6kIAuqeiBUxJ3nWTwNVE'},
 'traceId': '4f06d4b3-3f02-4cf1-af04-a478fdbab43a',
 'status_code': 201,
 'response_reason': 'Created'}

## 1.7

In [10]:
def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='CoachMcloone', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
            params = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = ''
                if params_dict:
                    payload['title'] += f'{"Human attention needed: " if params_dict["alert_human"]==True else "Chatbot response: "}'
                    payload['body'] = params_dict['response']
                else:
                    payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
                payload['title'] += f'Send message to contact {contactId}.'
                payload['assignedTo'] = os.environ['userId']
                if params_dict:
                    if params_dict.get('phone_number') != None:
                        payload['title'] += f' Phone number: {params_dict["phone_number"]}'
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations') | (endpoint == 'searchUsers') | (endpoint == 'getLocation'): 
            if endpoint=='searchConversations':
                endpoint_url = f'conversations/search'
            elif endpoint == 'searchUsers':
                endpoint_url = f'users/search'            
            elif endpoint == 'getContacts':
                endpoint_url = f'contacts/'
            else:
                endpoint_url = f'locations/{locationId}'
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        elif endpoint == 'getConversation':
            endpoint_url = f'conversations/{contactId if contactId else path_param}'
            request_type = 'GET'
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1.5
results_dict[iteration] = ghl_request(
    contactId=contactId, 
    endpoint='createTask', 
    params_dict=None,
    payload=None, 
    location='CoachMcloone'
)

# results_dict[iteration] = ghl_request(
#     contactId='Silvia Hua', 
#     endpoint='searchUsers', 
#     params_dict=params_dict,
#     payload=None, 
#     location='CoachMcloone'
# )
results_dict[iteration]

Status code 201: Created


{'task': {'id': 'B8il8OZgdoxLXi8F6gry',
  'title': 'Send message to contact 6kIAuqeiBUxJ3nWTwNVE.',
  'body': 'Test task via GHL API at 2024-02-10T06:55:54Z Pacific time',
  'assignedTo': 'qFootuYP7QMGCn6qXDXp',
  'dueDate': '2024-02-10T06:55:54.000Z',
  'completed': False,
  'contactId': '6kIAuqeiBUxJ3nWTwNVE'},
 'traceId': '7c7d05b0-1294-4422-9c4d-434a00c74c17',
 'status_code': 201,
 'response_reason': 'Created'}

# *End of Page*